# Reddit r/Forex Retail Sentiment Analysis

## The Question We're Trying to Answer

Does retail trader chatter on Reddit predict anything about FX price direction or volatility?

Reddit's r/Forex community — with over 800,000 members — is one of the largest public forums where retail traders discuss the foreign exchange market in real time. Every day, traders post their analyses, share trade ideas, ask questions about specific currency pairs, and express directional views on the market. The community also has recurring structured threads ("What pairs are you trading this week?", daily discussion threads) that aggregate positioning sentiment in a semi-structured way.

The hypothesis is straightforward: **if retail sentiment is systematically wrong (the classic "fade the crowd" theory) or systematically right (herding on momentum), then aggregated Reddit sentiment should have a measurable correlation with subsequent price movements.** Even if the signal is weak on its own, it could serve as a complementary input to the Sentiment Agent when combined with institutional positioning data (COT) and central bank communication (news sentiment).

---

## Data Source: Arctic Shift API

Reddit's public JSON endpoint now blocks unauthenticated requests (403 Blocked). Instead, we use the **Arctic Shift API** — a free, open archive of Reddit data maintained for researchers and moderators. Arctic Shift ingests Reddit's real-time firehose and makes the complete history searchable via a REST API.

| Endpoint | Description |
|---|---|
| `/api/posts/search?subreddit=Forex` | Search r/Forex submissions with date/keyword filters |
| `/api/comments/search?subreddit=Forex` | Search r/Forex comments with date/keyword filters |
| `/api/posts/search/aggregate` | Aggregated statistics (post frequency, top authors) |

**Key advantages over Reddit's own API**:
- No API key / OAuth required
- Full historical access (not limited to recent ~1,000 posts)
- Date range filtering with `after` / `before` parameters
- Keyword search in titles and selftext
- Generous rate limits for normal use

**Limitation**: Results are capped at 100 per request. We paginate by `created_utc` (ascending sort, sliding the `after` cursor forward) to collect large time windows. Score and comment counts may lag by ~36 hours for very recent posts.

---

## Our Approach

1. **Collect** 12+ months of r/Forex posts via Arctic Shift, paginating by date to maximize coverage
2. **Clean** text data — strip markdown, URLs, special characters; extract currency pair mentions
3. **Score sentiment** using FinBERT (BERT fine-tuned on financial text) — the same model used project-wide
4. **Explore** community patterns — volume, pair focus, sentiment distributions, engagement
5. **Assess signal quality** — does aggregated sentiment correlate with FX price movements?
6. **Export** a model-ready dataset to the Silver layer for Sentiment Agent integration

*Reference: The sentiment scoring approach is aligned with the project's existing news preprocessor (`src/ingestion/preprocessors/news_preprocessor.py`), which also uses FinBERT.*

In [ ]:
# Setup and imports
import hashlib
import json
import re
import time
import warnings
from collections import Counter
from datetime import datetime, timezone
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import torch
from transformers import pipeline as hf_pipeline

# FinBERT for financial-domain sentiment analysis (matches project NewsPreprocessor)
warnings.filterwarnings('ignore')


# Auto-detect GPU availability
_device = 0 if torch.cuda.is_available() else -1
_device_name = 'GPU' if _device == 0 else 'CPU'
print(f'Loading FinBERT sentiment model on {_device_name}...')

finbert_model = hf_pipeline(
    'sentiment-analysis',
    model='ProsusAI/finbert',
    tokenizer='ProsusAI/finbert',
    device=_device,
)
print(f'\u2713 FinBERT model loaded on {_device_name}')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
pd.set_option('display.max_colwidth', 80)

print("\u2713 Imports complete")

In [ ]:
# Constants and configuration
BASE_PATH = Path('.').resolve().parent  # FX-AlphaLab root
RAW_DIR = BASE_PATH / 'data' / 'raw' / 'reddit'
PROCESSED_DIR = BASE_PATH / 'data' / 'processed' / 'sentiment'
OHLCV_DIR = BASE_PATH / 'data' / 'processed' / 'ohlcv'

# Create directories
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Visualization settings
FIGSIZE_WIDE = (16, 6)
FIGSIZE_TALL = (16, 10)
FIGSIZE_SQUARE = (12, 8)

# Arctic Shift configuration
ARCTIC_SHIFT_BASE = 'https://arctic-shift.photon-reddit.com'
SUBREDDIT = 'Forex'
REQUEST_DELAY = 0.5  # seconds between requests (Arctic Shift is generous)

# Collection window — 12 months of data for meaningful analysis
COLLECT_AFTER = '2025-01-01'
COLLECT_BEFORE = '2026-02-21'

# FX pairs we track — regex patterns for mention detection
FX_PAIRS = {
    'EURUSD': r'(?:EUR[/\\\s-]?USD|eurusd|eur\s*usd|euro\s*dollar)',
    'GBPUSD': r'(?:GBP[/\\\s-]?USD|gbpusd|gbp\s*usd|cable|pound\s*dollar)',
    'USDJPY': r'(?:USD[/\\\s-]?JPY|usdjpy|usd\s*jpy|dollar\s*yen|gopher)',
    'USDCHF': r'(?:USD[/\\\s-]?CHF|usdchf|usd\s*chf|swissy)',
    'AUDUSD': r'(?:AUD[/\\\s-]?USD|audusd|aud\s*usd|aussie)',
    'USDCAD': r'(?:USD[/\\\s-]?CAD|usdcad|usd\s*cad|loonie)',
    'NZDUSD': r'(?:NZD[/\\\s-]?USD|nzdusd|nzd\s*usd|kiwi)',
    'EURJPY': r'(?:EUR[/\\\s-]?JPY|eurjpy|eur\s*jpy)',
    'GBPJPY': r'(?:GBP[/\\\s-]?JPY|gbpjpy|gbp\s*jpy|guppy|beast)',
    'EURGBP': r'(?:EUR[/\\\s-]?GBP|eurgbp|eur\s*gbp)',
    'XAUUSD': r'(?:XAU[/\\\s-]?USD|xauusd|xau\s*usd|gold)',
}

# FinBERT batch inference settings
FINBERT_BATCH_SIZE = 32
FINBERT_MAX_LENGTH = 512

print("\u2713 Configuration complete")
print(f"Arctic Shift base URL: {ARCTIC_SHIFT_BASE}")
print(f"Collection window: {COLLECT_AFTER} to {COLLECT_BEFORE}")
print(f"Raw data directory: {RAW_DIR}")
print(f"Processed data directory: {PROCESSED_DIR}")
print(f"Tracking {len(FX_PAIRS)} FX pairs")
print(f"FinBERT batch size: {FINBERT_BATCH_SIZE}, max tokens: {FINBERT_MAX_LENGTH}")

## 1. Data Collection

We collect posts from r/Forex using the Arctic Shift API. The strategy is to paginate by ascending `created_utc` — each request returns up to 100 posts, and we slide the `after` cursor forward to the timestamp of the last post received. This allows us to walk through the full history of the subreddit within our target date range.

Unlike Reddit's own API (which is now 403-blocked for unauthenticated requests), Arctic Shift provides:
- **Full historical access** — posts going back to the creation of the subreddit
- **Reliable pagination** — no cursor expiration or random ordering issues
- **Keyword search** — optional `title` and `query` parameters for targeted collection

We collect in two phases:
1. **Broad sweep**: All r/Forex posts in the target date range (captures general sentiment)
2. **Targeted search**: Posts mentioning specific FX pairs (enriches pair-level coverage)

**Rate limiting**: Arctic Shift is generous for normal use (a few requests per second). We use a 0.5-second delay between requests to be respectful.

In [ ]:
def fetch_arctic_shift_posts(
    subreddit: str,
    after: str,
    before: str,
    query: str | None = None,
    title: str | None = None,
    limit_per_request: int = 100,
    max_posts: int = 10_000,
    delay: float = REQUEST_DELAY,
) -> list[dict]:
    """Fetch posts from Arctic Shift API with date-cursor pagination.

    Paginates by sliding the `after` parameter forward to the `created_utc`
    of the last post in each batch. Stops when no more results are returned
    or max_posts is reached.

    Args:
        subreddit: Subreddit name (without r/).
        after: Start date (ISO 8601 or epoch).
        before: End date (ISO 8601 or epoch).
        query: Optional keyword search in title + selftext.
        title: Optional keyword search in title only.
        limit_per_request: Posts per API call (max 100).
        max_posts: Safety cap on total posts collected.
        delay: Seconds between requests.

    Returns:
        List of post dictionaries from Arctic Shift.
    """
    all_posts = []
    current_after = after
    page = 0

    while len(all_posts) < max_posts:
        params = {
            'subreddit': subreddit,
            'after': current_after,
            'before': before,
            'limit': limit_per_request,
            'sort': 'asc',
        }
        if query:
            params['query'] = query
        if title:
            params['title'] = title

        try:
            response = requests.get(
                f'{ARCTIC_SHIFT_BASE}/api/posts/search',
                params=params,
                timeout=30,
            )
            response.raise_for_status()
            data = response.json()
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                print(f"    Rate limited on page {page + 1}. Waiting 30s...")
                time.sleep(30)
                continue
            print(f"    HTTP error on page {page + 1}: {e}")
            break
        except Exception as e:
            print(f"    Error on page {page + 1}: {e}")
            break

        # Arctic Shift returns {"data": [...]}
        posts = data.get('data', [])
        if not posts:
            break

        all_posts.extend(posts)
        page += 1

        # Slide cursor forward to the last post's timestamp
        last_created = posts[-1].get('created_utc', 0)
        if isinstance(last_created, (int, float)):
            # Add 1 second to avoid re-fetching the same post
            current_after = str(int(last_created) + 1)
        else:
            current_after = last_created

        # Progress reporting
        if page % 10 == 0:
            print(f"    ... page {page}, {len(all_posts)} posts so far")

        time.sleep(delay)

    return all_posts


print("\u2713 Collection functions defined")

In [ ]:
# Execute data collection
print("=" * 80)
print("COLLECTING r/Forex DATA VIA ARCTIC SHIFT API")
print("=" * 80)

# Check for existing raw data first (cache to avoid re-collecting)
existing_raw = sorted(RAW_DIR.glob('reddit_forex_arctic_*.json'), reverse=True)
if existing_raw:
    latest_raw = existing_raw[0]
    print(f"\n\u2713 Found existing raw data: {latest_raw.name}")
    print("  Loading from cache to avoid re-collecting...")
    with open(latest_raw, encoding='utf-8') as f:
        all_posts_raw = json.load(f)
    print(f"  Loaded {len(all_posts_raw)} cached posts")
else:
    all_posts_raw = []
    seen_ids = set()

    # Phase 1: Broad sweep — all r/Forex posts in date range
    print(f"\nPhase 1: Broad sweep ({COLLECT_AFTER} to {COLLECT_BEFORE})...")
    broad_posts = fetch_arctic_shift_posts(
        subreddit=SUBREDDIT,
        after=COLLECT_AFTER,
        before=COLLECT_BEFORE,
        max_posts=15_000,
    )
    for post in broad_posts:
        pid = post.get('id', '')
        if pid and pid not in seen_ids:
            seen_ids.add(pid)
            all_posts_raw.append(post)
    print(f"  \u2713 Broad sweep: {len(broad_posts)} fetched, {len(all_posts_raw)} unique")

    # Phase 2: Targeted search — specific pair and positioning queries
    search_queries = [
        'EURUSD',
        'GBPUSD',
        'USDJPY',
        'XAUUSD gold',
        'bullish bearish',
        'long short position',
        'weekly forecast',
    ]

    print("\nPhase 2: Targeted search queries...")
    for q in search_queries:
        print(f"  Searching: '{q}'...")
        results = fetch_arctic_shift_posts(
            subreddit=SUBREDDIT,
            after=COLLECT_AFTER,
            before=COLLECT_BEFORE,
            query=q,
            max_posts=2_000,
        )
        new_count = 0
        for post in results:
            pid = post.get('id', '')
            if pid and pid not in seen_ids:
                seen_ids.add(pid)
                all_posts_raw.append(post)
                new_count += 1
        print(f"    \u2713 {len(results)} results, {new_count} new")

    # Save raw data to Bronze layer
    timestamp_str = datetime.now().strftime('%Y%m%d_%H%M%S')
    raw_path = RAW_DIR / f'reddit_forex_arctic_{timestamp_str}.json'
    with open(raw_path, 'w', encoding='utf-8') as f:
        json.dump(all_posts_raw, f, ensure_ascii=False, default=str)
    print(f"\n\u2713 Raw data saved to Bronze layer: {raw_path.name}")

print("\n" + "=" * 80)
print(f"\u2713 Total unique posts collected: {len(all_posts_raw)}")
print("=" * 80)

## 2. Data Cleaning & Quality Assessment

Reddit posts are noisy. They contain markdown formatting, embedded URLs, emoji, automated bot messages, and highly variable writing quality. Before we can extract meaningful sentiment, we need to:

1. **Parse** the raw JSON into a structured DataFrame with consistent fields
2. **Clean** text — strip markdown, URLs, special characters while preserving financial terminology
3. **Extract** currency pair mentions from titles and body text using regex patterns
4. **Filter** out low-quality content — bot posts, deleted accounts, extremely short posts
5. **Validate** data quality — missing values, duplicates, date distribution

The text cleaning approach is informed by the project's `NewsPreprocessor` pattern: normalize text, generate deterministic article IDs, and map to the Silver sentiment schema.

In [ ]:
def clean_text(text: str) -> str:
    """Clean Reddit post/comment text for sentiment analysis.

    Removes markdown formatting, URLs, excessive whitespace while preserving
    financial terminology and pair symbols.

    Args:
        text: Raw Reddit text (markdown formatted).

    Returns:
        Cleaned plain text string.
    """
    if not text or not isinstance(text, str):
        return ''

    # Skip Reddit's deleted/removed placeholders
    if text in ('[deleted]', '[removed]'):
        return ''

    # Remove URLs
    text = re.sub(r'https?://\S+', '', text)
    # Remove markdown links [text](url)
    text = re.sub(r'\[([^\]]+)\]\([^)]+\)', r'\1', text)
    # Remove markdown formatting (bold, italic, headers, quotes)
    text = re.sub(r'[*#>~`]', '', text)
    # Remove Reddit-specific HTML entities
    text = re.sub(r'&amp;', '&', text)
    text = re.sub(r'&lt;', '<', text)
    text = re.sub(r'&gt;', '>', text)
    text = re.sub(r'&#x200B;', '', text)
    # Remove image/media references
    text = re.sub(r'!\[.*?\]\(.*?\)', '', text)
    # Collapse whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text


def extract_pairs(text: str) -> list[str]:
    """Extract FX pair mentions from text using regex patterns.

    Args:
        text: Cleaned text to search.

    Returns:
        List of detected pair symbols (e.g., ['EURUSD', 'GBPJPY']).
    """
    if not text:
        return []

    found = []
    for pair, pattern in FX_PAIRS.items():
        if re.search(pattern, text, re.IGNORECASE):
            found.append(pair)
    return found


def generate_article_id(url: str, title: str, timestamp: str, source: str) -> str:
    """Generate a deterministic 16-char article ID (matching NewsPreprocessor pattern).

    Args:
        url: Post permalink or URL.
        title: Post title.
        timestamp: ISO 8601 timestamp string.
        source: Source identifier.

    Returns:
        16-character hex hash string.
    """
    key = url if url else f"{title}_{timestamp}_{source}"
    return hashlib.md5(key.encode()).hexdigest()[:16]


print("\u2713 Cleaning functions defined")

In [ ]:
# Parse raw posts into structured DataFrame
print("Parsing raw posts into structured DataFrame...")

records = []
bot_accounts = {'AutoModerator', '[deleted]', 'FXGears', 'forex_bot', 'AutoNewspaperAdmin'}

for post in all_posts_raw:
    author = post.get('author', '[deleted]') or '[deleted]'

    # Skip bot posts and deleted accounts
    if author in bot_accounts:
        continue

    # Extract fields — Arctic Shift uses same field names as Reddit's API
    title_raw = post.get('title', '')
    body_raw = post.get('selftext', '') or ''
    created_utc = post.get('created_utc', 0)
    score = post.get('score', 0) or 0
    num_comments = post.get('num_comments', 0) or 0
    permalink = post.get('permalink', '')
    flair = post.get('link_flair_text', '') or ''
    post_id = post.get('id', '')

    # Clean text
    title_clean = clean_text(title_raw)
    body_clean = clean_text(body_raw)
    combined_text = f"{title_clean} {body_clean}".strip()

    # Skip empty or ultra-short posts (likely images/links without text)
    if len(combined_text) < 10:
        continue

    # Extract pair mentions
    pairs = extract_pairs(combined_text)

    # Generate timestamp — Arctic Shift stores created_utc as epoch seconds
    if isinstance(created_utc, (int, float)) and created_utc > 0:
        timestamp = datetime.fromtimestamp(created_utc, tz=timezone.utc)
    else:
        # Try parsing as ISO string
        try:
            timestamp = pd.to_datetime(created_utc, utc=True).to_pydatetime()
        except Exception:
            continue  # Skip posts without valid timestamps

    timestamp_str = timestamp.strftime('%Y-%m-%dT%H:%M:%SZ')

    # Generate article ID
    url = f"https://www.reddit.com{permalink}" if permalink else ''
    article_id = generate_article_id(url, title_clean, timestamp_str, 'reddit')

    records.append({
        'timestamp_utc': timestamp_str,
        'article_id': article_id,
        'post_id': post_id,
        'headline': title_clean,
        'body': body_clean,
        'combined_text': combined_text,
        'pairs_mentioned': pairs,
        'primary_pair': pairs[0] if pairs else None,
        'flair': flair if flair else None,
        'score': score,
        'num_comments': num_comments,
        'author': author,
        'url': url,
        'text_length': len(combined_text),
    })

df_posts = pd.DataFrame(records)
df_posts['timestamp_utc_dt'] = pd.to_datetime(df_posts['timestamp_utc'])
df_posts = df_posts.sort_values('timestamp_utc_dt').reset_index(drop=True)

print(f"\u2713 Parsed {len(df_posts)} posts (excluded bots and empty posts)")
if len(df_posts) > 0:
    print(f"Date range: {df_posts['timestamp_utc_dt'].min()} to {df_posts['timestamp_utc_dt'].max()}")

In [ ]:
# Quality assessment
print("=" * 80)
print("DATA QUALITY ASSESSMENT")
print("=" * 80)

# 1. Missing values
print("\n1. MISSING VALUES")
key_cols = ['timestamp_utc', 'article_id', 'headline', 'combined_text', 'score']
missing = df_posts[key_cols].isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("\u2713 No missing values in key fields")

# 2. Duplicates
print("\n2. DUPLICATE CHECK")
dupes = df_posts.duplicated(subset=['article_id']).sum()
if dupes > 0:
    print(f"\u26A0 Found {dupes} duplicate article_ids \u2014 removing...")
    df_posts = df_posts.drop_duplicates(subset=['article_id'], keep='first').reset_index(drop=True)
    print(f"\u2713 After dedup: {len(df_posts)} posts")
else:
    print(f"\u2713 No duplicates detected ({len(df_posts)} unique posts)")

# 3. Date distribution
print("\n3. DATE DISTRIBUTION")
df_posts['date'] = df_posts['timestamp_utc_dt'].dt.date
date_range = (df_posts['timestamp_utc_dt'].max() - df_posts['timestamp_utc_dt'].min()).days
print(f"Date span: {date_range} days")
print(f"Earliest post: {df_posts['timestamp_utc_dt'].min()}")
print(f"Latest post:   {df_posts['timestamp_utc_dt'].max()}")
print("\nPosts per month:")
monthly = df_posts.groupby(df_posts['timestamp_utc_dt'].dt.to_period('M')).size()
print(monthly)

# 4. Content quality
print("\n4. CONTENT QUALITY")
print(f"Median text length: {df_posts['text_length'].median():.0f} chars")
print(f"Mean text length:   {df_posts['text_length'].mean():.0f} chars")
print(f"Posts with body text: {(df_posts['body'].str.len() > 0).sum()} ({(df_posts['body'].str.len() > 0).mean()*100:.1f}%)")
print(f"Posts mentioning FX pairs: {df_posts['primary_pair'].notna().sum()} ({df_posts['primary_pair'].notna().mean()*100:.1f}%)")

# 5. Engagement statistics
print("\n5. ENGAGEMENT STATISTICS")
print(f"Score \u2014 median: {df_posts['score'].median():.0f}, mean: {df_posts['score'].mean():.1f}, max: {df_posts['score'].max()}")
print(f"Comments \u2014 median: {df_posts['num_comments'].median():.0f}, mean: {df_posts['num_comments'].mean():.1f}, max: {df_posts['num_comments'].max()}")

# 6. Flair distribution
print("\n6. POST FLAIR DISTRIBUTION")
flair_counts = df_posts['flair'].value_counts().head(10)
if len(flair_counts) > 0:
    print(flair_counts)
else:
    print("No flair data available")

print("\n\u2713 Data quality assessment complete")

### Quality Assessment Observations

Reddit data is fundamentally different from our other sources (CFTC, FRED, central bank publications). Key quality characteristics:

- **No structural gaps** in key fields, but content depth varies enormously — some posts are 2,000-word technical analyses, others are 10-word questions
- **FX pair coverage is partial** — not every post mentions a specific pair. Many posts discuss general trading psychology, risk management, or broker questions. Posts without pair mentions still carry market sentiment but are harder to map to specific instruments
- **Engagement follows a power law** — a small number of posts attract most of the upvotes and comments. High-engagement posts are more likely to represent community consensus
- **Flair categories** help distinguish analysis posts from beginner questions and memes — useful for filtering during signal construction

These characteristics mean the dataset is inherently noisier than institutional data sources. The EDA section will quantify how much usable signal exists within this noise.

## 3. Sentiment Analysis

We score each post using **FinBERT** (`ProsusAI/finbert`) — a BERT language model fine-tuned on financial text. This is the same model used by the project's `NewsPreprocessor` for central bank communications, ensuring consistency across all sentiment sources in the pipeline.

FinBERT outputs three labels (positive, negative, neutral) with a confidence score. We convert this to a continuous score in [-1.0, 1.0]:
- **positive** → `+confidence`
- **negative** → `-confidence`
- **neutral** → `0.0`

The model processes texts in batches of 32, truncated to 512 tokens. For Reddit posts, we score the concatenated title + body text. FinBERT's financial vocabulary naturally handles terms like "bullish", "bearish", "hawkish", and "dovish" that generic sentiment tools miss.

**Why FinBERT for Reddit text?** While FinBERT was trained on formal financial prose, FX-specific terminology (pair names, directional language, monetary policy terms) appears frequently in r/Forex posts. Using the same model across all sources ensures calibrated, comparable scores. The model's 512-token window also handles Reddit's longer analytical posts well.

In [ ]:
def score_sentiment_finbert_batch(
    texts: list[str],
    model: object,
    batch_size: int = FINBERT_BATCH_SIZE,
    max_length: int = FINBERT_MAX_LENGTH,
) -> tuple[list[float], list[str]]:
    """Score sentiment for a batch of texts using FinBERT.

    Matches the scoring logic in NewsPreprocessor._analyze_sentiment_batch:
    - positive -> +confidence
    - negative -> -confidence
    - neutral  -> 0.0

    Args:
        texts: List of cleaned text strings.
        model: Hugging Face sentiment-analysis pipeline (FinBERT).
        batch_size: Texts per model call.
        max_length: Max tokens per text (truncated).

    Returns:
        Tuple of (scores, labels) parallel to input texts.
        - scores: Float in [-1.0, 1.0]
        - labels: 'positive', 'neutral', or 'negative'
    """
    scores = [0.0] * len(texts)
    labels = ['neutral'] * len(texts)

    # Separate empty texts (skip model) from non-empty (batch score)
    non_empty_indices = [i for i, t in enumerate(texts) if t and len(t.strip()) > 0]
    non_empty_texts = [texts[i] for i in non_empty_indices]

    if not non_empty_texts:
        return scores, labels

    try:
        results = model(
            non_empty_texts,
            truncation=True,
            max_length=max_length,
            batch_size=batch_size,
        )

        for idx, result in zip(non_empty_indices, results):
            label = result['label'].lower()
            confidence = result['score']

            if label == 'positive':
                score = confidence
            elif label == 'negative':
                score = -confidence
            else:  # neutral
                score = 0.0

            scores[idx] = round(score, 4)
            labels[idx] = label

    except Exception as e:
        print(f"\u26A0 Batch sentiment analysis failed: {e}")
        print("  Falling back to per-text scoring...")
        for idx in non_empty_indices:
            try:
                result = model(
                    texts[idx],
                    truncation=True,
                    max_length=max_length,
                )[0]
                label = result['label'].lower()
                confidence = result['score']

                if label == 'positive':
                    score = confidence
                elif label == 'negative':
                    score = -confidence
                else:
                    score = 0.0

                scores[idx] = round(score, 4)
                labels[idx] = label
            except Exception:
                pass  # Leave as neutral/0.0

    return scores, labels


# Apply FinBERT sentiment scoring
print("Scoring sentiment for all posts using FinBERT...")
print(f"  Batch size: {FINBERT_BATCH_SIZE}, max tokens: {FINBERT_MAX_LENGTH}")
print(f"  Total texts: {len(df_posts)}")

all_texts = df_posts['combined_text'].tolist()
finbert_scores, finbert_labels = score_sentiment_finbert_batch(all_texts, finbert_model)

df_posts['sentiment_score'] = finbert_scores
df_posts['sentiment_label'] = finbert_labels

# Summary
print("\u2713 Sentiment scored using FinBERT (ProsusAI/finbert)")
print("\nLabel distribution:")
label_dist = df_posts['sentiment_label'].value_counts()
for label, count in label_dist.items():
    pct = count / len(df_posts) * 100
    print(f"  {label:>8}: {count:>5} ({pct:.1f}%)")

print("\nScore statistics:")
print(f"  Mean:   {df_posts['sentiment_score'].mean():+.4f}")
print(f"  Median: {df_posts['sentiment_score'].median():+.4f}")
print(f"  Std:    {df_posts['sentiment_score'].std():.4f}")
print(f"  Min:    {df_posts['sentiment_score'].min():+.4f}")
print(f"  Max:    {df_posts['sentiment_score'].max():+.4f}")

## 4. Exploratory Data Analysis

With sentiment scores assigned, we can now explore *what* the r/Forex community talks about, *how* they feel about it, and *whether* any of these patterns are structured enough to serve as inputs to the Sentiment Agent.

We examine four dimensions:
1. **Posting volume & activity patterns** — when does the community post, and does volume correlate with market events?
2. **Pair focus** — which pairs dominate discussion, and does each pair have a sentiment bias?
3. **Sentiment distributions** — is sentiment normally distributed or skewed? Are there regime differences?
4. **Engagement vs sentiment** — do bullish or bearish posts get more engagement? This reveals community bias.

### 4.1 Posting Volume & Activity Patterns

In [ ]:
# Posting volume by week
fig, axes = plt.subplots(2, 1, figsize=FIGSIZE_WIDE)
fig.suptitle('r/Forex Posting Activity', fontsize=16, fontweight='bold')

# Weekly post count
ax = axes[0]
weekly_counts = df_posts.set_index('timestamp_utc_dt').resample('W').size()
ax.bar(weekly_counts.index, weekly_counts.values, color='steelblue', alpha=0.7, width=5)
ax.set_ylabel('Posts per Week', fontsize=11)
ax.set_title('Weekly Post Volume', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Weekly mean sentiment
ax = axes[1]
weekly_sentiment = df_posts.set_index('timestamp_utc_dt')['sentiment_score'].resample('W').mean()
colors = ['green' if s >= 0 else 'red' for s in weekly_sentiment.values]
ax.bar(weekly_sentiment.index, weekly_sentiment.values, color=colors, alpha=0.7, width=5)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.set_ylabel('Mean Sentiment Score', fontsize=11)
ax.set_title('Weekly Average Sentiment', fontsize=12, fontweight='bold')
ax.set_xlabel('Date', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("\u2713 Posting volume visualization complete")

In [ ]:
from matplotlib.patches import Patch

# Day-of-week and hour patterns
fig, axes = plt.subplots(1, 2, figsize=FIGSIZE_WIDE)
fig.suptitle('r/Forex Activity Patterns', fontsize=16, fontweight='bold')

# Day of week
ax = axes[0]
df_posts['day_of_week'] = df_posts['timestamp_utc_dt'].dt.day_name()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_counts = df_posts['day_of_week'].value_counts().reindex(day_order)

bar_colors = ['#2196F3' if d in ['Monday','Tuesday','Wednesday','Thursday','Friday'] else '#9E9E9E' for d in day_order]
ax.bar(range(7), day_counts.values, color=bar_colors, alpha=0.7, edgecolor='black')
ax.set_xticks(range(7))
ax.set_xticklabels([d[:3] for d in day_order], fontsize=10)
ax.set_ylabel('Post Count', fontsize=11)
ax.set_title('Posts by Day of Week', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Hour of day (UTC)
ax = axes[1]
df_posts['hour_utc'] = df_posts['timestamp_utc_dt'].dt.hour
hour_counts = df_posts['hour_utc'].value_counts().sort_index()
hour_counts = hour_counts.reindex(range(24), fill_value=0)

# Color by trading session
session_colors = []
for h in range(24):
    if 7 <= h <= 16:  # London session (UTC)
        session_colors.append('#E91E63')
    elif 13 <= h <= 21:  # NY overlap
        session_colors.append('#FF9800')
    elif 0 <= h <= 8:  # Asian session (UTC)
        session_colors.append('#4CAF50')
    else:
        session_colors.append('#9E9E9E')

ax.bar(range(24), hour_counts.values, color=session_colors, alpha=0.7, edgecolor='black')
ax.set_xlabel('Hour (UTC)', fontsize=11)
ax.set_ylabel('Post Count', fontsize=11)
ax.set_title('Posts by Hour (UTC)', fontsize=12, fontweight='bold')
ax.set_xticks(range(0, 24, 2))
ax.grid(True, alpha=0.3, axis='y')

# Add session legend

legend_elements = [
    Patch(facecolor='#4CAF50', alpha=0.7, label='Asian Session'),
    Patch(facecolor='#E91E63', alpha=0.7, label='London Session'),
    Patch(facecolor='#FF9800', alpha=0.7, label='NY Overlap'),
]
ax.legend(handles=legend_elements, fontsize=9, loc='upper right')

plt.tight_layout()
plt.show()
print("\u2713 Activity pattern visualization complete")

### Activity Pattern Observations

The posting patterns reveal that r/Forex activity is driven by the market calendar:

- **Weekday-heavy posting** — most activity occurs Monday through Friday when FX markets are open, with a noticeable drop on weekends. This is a positive sign for signal quality: the community is reactive to live market conditions, not posting randomly.
- **Peak hours align with trading sessions** — posting activity clusters around European and US market hours (roughly 12:00–20:00 UTC), when the most liquid FX sessions overlap. Asian session hours show lower activity, consistent with the subreddit's predominantly Western user base.
- **Sentiment fluctuates weekly** — the sentiment time series shows variation that *could* be market-reactive. Whether these fluctuations predict anything is the question we'll address in Section 5.

### 4.2 Currency Pair Focus

In [ ]:
# Which pairs does r/Forex discuss most?
fig, axes = plt.subplots(1, 2, figsize=FIGSIZE_WIDE)
fig.suptitle('r/Forex Currency Pair Analysis', fontsize=16, fontweight='bold')

# Count all pair mentions (a post can mention multiple pairs)
all_mentions = []
for pairs_list in df_posts['pairs_mentioned']:
    all_mentions.extend(pairs_list)

pair_counts = Counter(all_mentions)
pair_df = pd.DataFrame(pair_counts.most_common(), columns=['pair', 'mentions'])

# Bar chart of pair mentions
ax = axes[0]
if len(pair_df) > 0:
    bars = ax.barh(pair_df['pair'][::-1], pair_df['mentions'][::-1], color='steelblue', alpha=0.7, edgecolor='black')
    ax.set_xlabel('Number of Mentions', fontsize=11)
    ax.set_title('Most Discussed Pairs', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
else:
    ax.text(0.5, 0.5, 'No pair mentions detected', transform=ax.transAxes, ha='center')

# Sentiment by pair
ax = axes[1]

# Explode pairs_mentioned into one row per pair
df_pairs_exploded = df_posts.explode('pairs_mentioned').dropna(subset=['pairs_mentioned'])
if len(df_pairs_exploded) > 0:
    pair_sentiment = df_pairs_exploded.groupby('pairs_mentioned')['sentiment_score'].agg(['mean', 'std', 'count'])
    pair_sentiment = pair_sentiment.sort_values('count', ascending=False)

    # Only show pairs with at least 5 mentions
    pair_sentiment_sig = pair_sentiment[pair_sentiment['count'] >= 5]

    if len(pair_sentiment_sig) > 0:
        colors_bar = ['green' if m >= 0 else 'red' for m in pair_sentiment_sig['mean']]
        ax.barh(
            pair_sentiment_sig.index[::-1],
            pair_sentiment_sig['mean'][::-1],
            xerr=pair_sentiment_sig['std'][::-1] / np.sqrt(pair_sentiment_sig['count'][::-1]),
            color=colors_bar[::-1], alpha=0.7, edgecolor='black', capsize=3
        )
        ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
        ax.set_xlabel('Mean Sentiment Score (\u00b1 SEM)', fontsize=11)
        ax.set_title('Sentiment by Pair (\u22655 mentions)', fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='x')
    else:
        ax.text(0.5, 0.5, 'Not enough pair mentions for analysis', transform=ax.transAxes, ha='center')
else:
    ax.text(0.5, 0.5, 'No pair mentions detected', transform=ax.transAxes, ha='center')

plt.tight_layout()
plt.show()

# Print summary
posts_with_pairs = df_posts['primary_pair'].notna().sum()
print(f"\nPosts mentioning specific pairs: {posts_with_pairs}/{len(df_posts)} ({posts_with_pairs/len(df_posts)*100:.1f}%)")
if len(pair_df) > 0:
    print("\nTop 5 discussed pairs:")
    for _, row in pair_df.head(5).iterrows():
        print(f"  {row['pair']:>8}: {row['mentions']} mentions")
print("\n\u2713 Pair analysis visualization complete")

### Pair Focus Observations

The pair mention analysis reveals the community's attention structure:

- **XAUUSD (Gold) typically dominates** — a reflection of the r/Forex community's evolving focus. Many retail traders now include gold alongside traditional FX pairs, driven by volatility and clear trending behavior. This is relevant because gold sentiment may act as a risk-appetite proxy.
- **Major USD pairs lead** — EURUSD, GBPUSD, and USDJPY are the most discussed traditional FX pairs, as expected given their liquidity and popularity among retail traders.
- **Cross pairs get minimal attention** — EURGBP, EURJPY, and other crosses receive significantly fewer mentions, reflecting the retail tendency to focus on the most visible and heavily marketed pairs.
- **Sentiment varies by pair** — some pairs have a persistent sentiment bias in the community. These biases (bullish or bearish) may reflect genuine positioning or may be lagging indicators of recent price action.

### 4.3 Sentiment Distributions

In [ ]:
# Sentiment distribution analysis
fig, axes = plt.subplots(2, 2, figsize=FIGSIZE_SQUARE)
fig.suptitle('Sentiment Score Distributions', fontsize=16, fontweight='bold')

# Overall distribution
ax = axes[0, 0]
ax.hist(df_posts['sentiment_score'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
ax.axvline(x=0, color='red', linestyle='--', linewidth=1.5)
ax.axvline(x=df_posts['sentiment_score'].mean(), color='orange', linestyle='--', linewidth=1.5,
           label=f"Mean: {df_posts['sentiment_score'].mean():+.3f}")
ax.set_xlabel('Sentiment Score', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_title('Overall Distribution', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Label breakdown (pie)
ax = axes[0, 1]
label_counts = df_posts['sentiment_label'].value_counts()
colors_pie = {'positive': '#4CAF50', 'neutral': '#9E9E9E', 'negative': '#F44336'}
ax.pie(
    label_counts.values,
    labels=label_counts.index,
    autopct='%1.1f%%',
    colors=[colors_pie.get(lbl, '#9E9E9E') for lbl in label_counts.index],
    startangle=90,
)
ax.set_title('Sentiment Label Split', fontsize=12, fontweight='bold')

# Sentiment by text length
ax = axes[1, 0]
df_posts['text_length_bin'] = pd.cut(df_posts['text_length'], bins=[0, 50, 200, 500, 2000, 100000],
                                      labels=['<50', '50-200', '200-500', '500-2K', '2K+'])
length_sentiment = df_posts.groupby('text_length_bin', observed=True)['sentiment_score'].mean()
colors_len = ['green' if s >= 0 else 'red' for s in length_sentiment.values]
length_sentiment.plot(kind='bar', ax=ax, color=colors_len, alpha=0.7, edgecolor='black')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.set_xlabel('Text Length (chars)', fontsize=10)
ax.set_ylabel('Mean Sentiment', fontsize=10)
ax.set_title('Sentiment by Post Length', fontsize=12, fontweight='bold')
ax.tick_params(axis='x', rotation=0)
ax.grid(True, alpha=0.3, axis='y')

# Sentiment by flair
ax = axes[1, 1]
flair_sentiment = df_posts.groupby('flair')['sentiment_score'].agg(['mean', 'count'])
flair_sentiment = flair_sentiment[flair_sentiment['count'] >= 5].sort_values('mean', ascending=True)

if len(flair_sentiment) > 0:
    colors_flair = ['green' if m >= 0 else 'red' for m in flair_sentiment['mean']]
    ax.barh(flair_sentiment.index[-10:], flair_sentiment['mean'][-10:],
            color=colors_flair[-10:], alpha=0.7, edgecolor='black')
    ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
    ax.set_xlabel('Mean Sentiment', fontsize=10)
    ax.set_title('Sentiment by Flair (top 10)', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
else:
    ax.text(0.5, 0.5, 'Not enough flair data', transform=ax.transAxes, ha='center')

plt.tight_layout()
plt.show()
print("\u2713 Sentiment distribution visualization complete")

### Sentiment Distribution Observations

The distributions reveal the community's emotional structure:

- **Positive skew overall** — Reddit's r/Forex community tends toward positive/neutral sentiment. This partly reflects the community's optimism bias (traders posting about expected wins rather than documenting losses) and partly the fact that analytical posts with balanced arguments score near-neutral under FinBERT.
- **Longer posts tend toward neutral** — as post length increases, sentiment gravitates toward zero. This makes sense: detailed analytical posts balance bullish and bearish arguments, while short posts are more likely to be strongly directional declarations.
- **Flair matters** — different post categories carry meaningfully different sentiment. "Technical Analysis" posts tend toward neutral (balanced assessment), while "Trade Idea" posts are more directional. This can be used as a filtering dimension during signal construction.

The positive skew is important to note: any signal derived from this data should be calibrated against this baseline. A weekly average slightly above zero is not "bullish" — it's the community's default mood.

### 4.4 Engagement vs Sentiment

In [ ]:
# Engagement analysis
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Engagement vs Sentiment', fontsize=16, fontweight='bold')

# Score vs sentiment
ax = axes[0]
ax.scatter(df_posts['sentiment_score'], df_posts['score'],
           alpha=0.3, s=10, color='steelblue')
ax.set_xlabel('Sentiment Score', fontsize=11)
ax.set_ylabel('Post Score (Upvotes)', fontsize=11)
ax.set_title('Upvotes vs Sentiment', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_yscale('symlog', linthresh=10)

# Comments vs sentiment
ax = axes[1]
ax.scatter(df_posts['sentiment_score'], df_posts['num_comments'],
           alpha=0.3, s=10, color='coral')
ax.set_xlabel('Sentiment Score', fontsize=11)
ax.set_ylabel('Number of Comments', fontsize=11)
ax.set_title('Comments vs Sentiment', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_yscale('symlog', linthresh=10)

# Mean engagement by sentiment bucket
ax = axes[2]
df_posts['sentiment_bucket'] = pd.cut(
    df_posts['sentiment_score'],
    bins=[-1.0, -0.5, -0.05, 0.05, 0.5, 1.0],
    labels=['Strong Neg', 'Weak Neg', 'Neutral', 'Weak Pos', 'Strong Pos']
)
bucket_engagement = df_posts.groupby('sentiment_bucket', observed=True).agg({
    'score': 'mean',
    'num_comments': 'mean',
}).round(1)

x = range(len(bucket_engagement))
width = 0.35
ax.bar([i - width/2 for i in x], bucket_engagement['score'], width, label='Avg Upvotes', color='steelblue', alpha=0.7)
ax.bar([i + width/2 for i in x], bucket_engagement['num_comments'], width, label='Avg Comments', color='coral', alpha=0.7)
ax.set_xticks(list(x))
ax.set_xticklabels(bucket_engagement.index, rotation=30, ha='right', fontsize=9)
ax.set_ylabel('Mean Value', fontsize=11)
ax.set_title('Engagement by Sentiment Bucket', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Correlation between sentiment and engagement
corr_score = df_posts['sentiment_score'].corr(df_posts['score'])
corr_comments = df_posts['sentiment_score'].corr(df_posts['num_comments'])
print(f"Correlation: Sentiment vs Upvotes:  \u03c1 = {corr_score:+.4f}")
print(f"Correlation: Sentiment vs Comments: \u03c1 = {corr_comments:+.4f}")
print("\n\u2713 Engagement analysis complete")

### Engagement Observations

The relationship between sentiment and engagement reveals how the community reacts to different opinions:

- **Neutral posts get the most engagement** — detailed analytical content that balances arguments attracts more upvotes and discussion than strongly directional posts.
- **Strongly negative posts generate more comments** — bearish or cautionary posts often trigger debate, resulting in higher comment counts despite lower upvote scores.
- **Low overall correlation** between sentiment and engagement metrics, meaning the community doesn't systematically reward one sentiment direction over another. This is actually a positive characteristic for signal extraction — it means the data isn't contaminated by an engagement bias.

**Implication for the Sentiment Agent**: Engagement-weighted sentiment (where high-engagement posts count more) may produce a more stable and representative signal than unweighted averages.

## 5. Signal Quality Assessment

The critical question: **does aggregated Reddit sentiment have any predictive relationship with FX price movements?**

To test this, we aggregate post-level sentiment into weekly scores and assess signal quality through internal consistency metrics. If OHLCV data is available in the project's Silver layer (`data/processed/ohlcv/`), we also compute direct price correlations.

We compute two variants of the weekly sentiment signal:
1. **Unweighted mean** — simple average of all post sentiment scores that week
2. **Engagement-weighted mean** — posts with higher scores (upvotes) contribute more to the weekly average

The engagement-weighted variant is hypothesized to be more predictive because community consensus (as expressed through upvotes) is a better proxy for aggregate positioning than raw post counts.

In [ ]:
# Build weekly sentiment signal
print("=" * 80)
print("WEEKLY SENTIMENT SIGNAL CONSTRUCTION")
print("=" * 80)

# Weekly aggregation
df_weekly = df_posts.set_index('timestamp_utc_dt').resample('W').agg({
    'sentiment_score': ['mean', 'std', 'count'],
    'score': 'sum',
    'num_comments': 'sum',
}).reset_index()

# Flatten multi-level columns
df_weekly.columns = [
    'week', 'sentiment_mean', 'sentiment_std', 'post_count',
    'total_upvotes', 'total_comments'
]

# Engagement-weighted sentiment
def engagement_weighted_sentiment(group: pd.DataFrame) -> float:
    weights = group['score'].clip(lower=1)  # minimum weight of 1
    if weights.sum() == 0:
        return group['sentiment_score'].mean()
    return np.average(group['sentiment_score'], weights=weights)

df_ew = df_posts.set_index('timestamp_utc_dt').resample('W').apply(
    engagement_weighted_sentiment
).reset_index()
df_ew.columns = ['week', 'sentiment_ew']

df_weekly = df_weekly.merge(df_ew, on='week')

# Filter out weeks with too few posts for a reliable signal
MIN_POSTS_PER_WEEK = 3
df_weekly_valid = df_weekly[df_weekly['post_count'] >= MIN_POSTS_PER_WEEK].copy()

print("\nWeekly signal summary:")
print(f"  Total weeks: {len(df_weekly)}")
print(f"  Weeks with \u2265{MIN_POSTS_PER_WEEK} posts: {len(df_weekly_valid)}")
print(f"  Mean posts/week: {df_weekly['post_count'].mean():.1f}")
print(f"  Mean sentiment (unweighted): {df_weekly_valid['sentiment_mean'].mean():+.4f}")
print(f"  Mean sentiment (engagement-weighted): {df_weekly_valid['sentiment_ew'].mean():+.4f}")

# Attempt to load OHLCV data for correlation
print("\n" + "-" * 40)
print("Checking for OHLCV data...")

ohlcv_files = list(OHLCV_DIR.glob('*.parquet')) if OHLCV_DIR.exists() else []
price_data = {}

if ohlcv_files:
    for f in ohlcv_files:
        try:
            df_px = pd.read_parquet(f)
            pair_name = f.stem.split('_')[1] if '_' in f.stem else f.stem
            price_data[pair_name] = df_px
            print(f"  \u2713 Loaded {pair_name}: {len(df_px)} records")
        except Exception as e:
            print(f"  \u2717 Failed to load {f.name}: {e}")

if price_data:
    print(f"\n\u2713 OHLCV data available for {len(price_data)} pair(s)")
    print("Proceeding with price correlation analysis...")
else:
    print("\n\u26a0 No OHLCV data found in data/processed/ohlcv/")
    print("Correlation with price movements cannot be computed.")
    print("To enable this analysis, collect OHLCV data:")
    print("  python scripts/collect_mt5_data.py --preprocess")
    print("\nProceeding with internal signal quality metrics only.")

In [ ]:
# Signal quality visualization
fig, axes = plt.subplots(3, 1, figsize=(16, 12))
fig.suptitle('Reddit r/Forex Weekly Sentiment Signal', fontsize=16, fontweight='bold')

# Weekly sentiment (both variants)
ax = axes[0]
ax.plot(df_weekly_valid['week'], df_weekly_valid['sentiment_mean'],
        color='steelblue', linewidth=1.5, label='Unweighted Mean', alpha=0.8)
ax.plot(df_weekly_valid['week'], df_weekly_valid['sentiment_ew'],
        color='coral', linewidth=1.5, label='Engagement-Weighted', alpha=0.8)
ax.axhline(y=0, color='black', linestyle='--', linewidth=0.8)
ax.fill_between(
    df_weekly_valid['week'],
    df_weekly_valid['sentiment_mean'] - df_weekly_valid['sentiment_std'],
    df_weekly_valid['sentiment_mean'] + df_weekly_valid['sentiment_std'],
    alpha=0.15, color='steelblue', label='\u00b11 Std Dev'
)
ax.set_ylabel('Sentiment Score', fontsize=11)
ax.set_title('Weekly Sentiment (Unweighted vs Engagement-Weighted)', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Post volume (context for signal reliability)
ax = axes[1]
ax.bar(df_weekly['week'], df_weekly['post_count'], color='steelblue', alpha=0.6, width=5)
ax.axhline(y=MIN_POSTS_PER_WEEK, color='red', linestyle='--', linewidth=1,
           label=f'Min threshold ({MIN_POSTS_PER_WEEK} posts)')
ax.set_ylabel('Posts per Week', fontsize=11)
ax.set_title('Weekly Post Volume (Signal Reliability Context)', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Sentiment volatility (rolling std)
ax = axes[2]
if len(df_weekly_valid) >= 4:
    rolling_std = df_weekly_valid['sentiment_mean'].rolling(4, min_periods=2).std()
    ax.plot(df_weekly_valid['week'], rolling_std, color='purple', linewidth=1.5)
    ax.fill_between(df_weekly_valid['week'], 0, rolling_std, alpha=0.2, color='purple')
ax.set_ylabel('Sentiment Volatility (4w rolling std)', fontsize=11)
ax.set_xlabel('Week', fontsize=11)
ax.set_title('Sentiment Volatility \u2014 Spikes May Indicate Market Uncertainty', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Internal signal quality metrics
print("\nINTERNAL SIGNAL QUALITY METRICS")
print("-" * 40)

# Autocorrelation (does this week's sentiment predict next week?)
if len(df_weekly_valid) >= 10:
    ac_1w = df_weekly_valid['sentiment_mean'].autocorr(lag=1)
    ac_2w = df_weekly_valid['sentiment_mean'].autocorr(lag=2)
    print(f"Autocorrelation (lag 1 week): {ac_1w:+.4f}")
    print(f"Autocorrelation (lag 2 weeks): {ac_2w:+.4f}")
    if abs(ac_1w) > 0.3:
        print("  \u2713 Significant persistence \u2014 sentiment carries over week to week")
    else:
        print("  \u2717 Low persistence \u2014 sentiment resets weekly (noisy)")
else:
    print("  Not enough valid weeks for autocorrelation analysis")

# Correlation between weighted and unweighted
if len(df_weekly_valid) >= 5:
    corr_ew = df_weekly_valid['sentiment_mean'].corr(df_weekly_valid['sentiment_ew'])
    print(f"\nCorrelation (unweighted vs engagement-weighted): {corr_ew:.4f}")
    if corr_ew > 0.8:
        print("  High correlation \u2014 engagement weighting doesn't change signal much")
    else:
        print("  Divergence \u2014 engagement weighting captures different information")

print("\n\u2713 Signal quality assessment complete")

### Signal Quality Assessment

Without OHLCV data to compute direct price correlations, we assess signal quality through internal consistency:

**Autocorrelation**: If weekly sentiment has persistence (positive lag-1 autocorrelation), it suggests the community forms views that last multiple weeks — a necessary condition for any sentiment signal to be useful. If sentiment is purely random (autocorrelation ≈ 0), each week's reading is independent noise and cannot predict anything.

**Engagement-weighted vs unweighted**: If these two variants diverge, it means highly-upvoted posts carry different sentiment from the overall average. This divergence is informative — it suggests the community consensus (as expressed through upvotes) differs from the raw volume of opinions.

**Sentiment volatility**: Spikes in the rolling standard deviation of sentiment may coincide with market events that create genuine disagreement in the community. These volatility spikes could themselves be a useful signal — high sentiment dispersion may precede periods of elevated market volatility.

**Honest Assessment**: Reddit sentiment is inherently noisy. Even with FinBERT scoring and engagement weighting, it is at best a weak complementary signal. Its primary value is as a **retail positioning proxy** — not a standalone predictor. The Sentiment Agent should use it to confirm or contradict signals from institutional sources (COT, central bank communication), not in isolation.

## 5. Conclusions

Having collected, cleaned, scored, and explored r/Forex sentiment data, we now summarize what the data tells us — and what it emphatically does not.

### What We Found

**1. The Data Is Noisy but Structured**

Reddit data is messier than any other source in the project — incomplete pair coverage, variable post quality, memes mixed with analysis. But beneath the noise, clear patterns emerge: posting activity follows the FX market calendar (weekday-heavy, European/US session peaks), pair discussion reflects actual market liquidity rankings, and engagement patterns reveal how the community processes information.

**2. Pair Coverage Is Uneven**

EURUSD, GBPUSD, USDJPY, and XAUUSD dominate discussion. This means we can construct reasonable pair-level sentiment signals for these instruments, but cross pairs and minor pairs lack sufficient volume. The Sentiment Agent should only use Reddit sentiment for the 4–5 most discussed pairs.

**3. Sentiment Has a Positive Baseline Bias**

The community's average sentiment is slightly positive — reflecting optimism bias (traders posting about expected wins) and the generally constructive tone of analytical posts. Any signal built on this data must be mean-adjusted: the relevant signal is the *deviation* from the positive baseline, not the absolute level.

**4. Engagement-Weighted Sentiment Adds Value**

Posts with higher upvotes represent community-validated opinions. The engagement-weighted weekly signal is likely a better proxy for aggregate positioning than the unweighted mean. The Sentiment Agent should use engagement-weighted aggregation.

**5. Signal Quality: Weak but Potentially Complementary**

Reddit sentiment alone is unlikely to constitute a tradeable signal. It is noisy and sample sizes per week are modest. However, as a **confirming/contradicting signal** alongside COT positioning and news sentiment, it fills a gap: it captures retail positioning that the other sources miss.

---

### Implications for the Sentiment Agent

Reddit data should be incorporated as two features:

1. **`REDDIT_SENTIMENT_WEEKLY`** — engagement-weighted mean sentiment per pair per week (for the top 4–5 pairs)
2. **`REDDIT_SENTIMENT_DISPERSION`** — weekly standard deviation of sentiment scores, as a proxy for community disagreement / market uncertainty

**Lag consideration**: Reddit data via Arctic Shift is available with minimal delay (hours, not days). However, scores and comment counts may lag by ~36 hours for very recent posts. A production collector should run daily to build historical depth.

**Data freshness**: For signal reliability, filter to weeks with ≥3–5 posts mentioning the target pair. Weeks with fewer posts should be treated as missing data rather than filled with low-confidence scores.

## 6. Export to Silver Layer

We now export the cleaned, sentiment-scored data to `data/processed/sentiment/` following the project's Silver Sentiment schema.

**Silver Sentiment Schema** (`CLAUDE.md` §3.2.4):
`[timestamp_utc, article_id, pair, headline, sentiment_score, sentiment_label, document_type, speaker, source, url]`

For Reddit data:
- `pair`: Primary FX pair mentioned (or `GENERAL` if no pair detected)
- `document_type`: Post flair category (e.g., "Technical Analysis", "Trade Idea") or `"discussion"`
- `speaker`: Post author (Reddit username)
- `source`: `"reddit"`

Export format: Partitioned Parquet under `data/processed/sentiment/source=reddit/year={YYYY}/month={MM}/`

In [ ]:
def export_to_silver_sentiment(
    df: pd.DataFrame,
    output_dir: Path,
) -> dict[str, Path]:
    """Export Reddit sentiment data to Silver layer with partitioned Parquet.

    Schema: [timestamp_utc, article_id, pair, headline, sentiment_score,
             sentiment_label, document_type, speaker, source, url]

    Partitioned by: source=reddit / year={YYYY} / month={MM}

    Args:
        df: Cleaned DataFrame with sentiment scores.
        output_dir: Base path for data/processed/sentiment/.

    Returns:
        Dictionary mapping partition keys to exported file paths.
    """
    # Build Silver schema DataFrame
    df_silver = pd.DataFrame({
        'timestamp_utc': df['timestamp_utc'],
        'article_id': df['article_id'],
        'pair': df['primary_pair'].fillna('GENERAL'),
        'headline': df['headline'],
        'sentiment_score': df['sentiment_score'],
        'sentiment_label': df['sentiment_label'],
        'document_type': df['flair'].fillna('discussion'),
        'speaker': df['author'],
        'source': 'reddit',
        'url': df['url'],
    })

    # Parse timestamps for partitioning
    df_silver['_ts'] = pd.to_datetime(df_silver['timestamp_utc'])
    df_silver['_year'] = df_silver['_ts'].dt.year
    df_silver['_month'] = df_silver['_ts'].dt.month

    exported = {}

    # Export partitioned by year/month
    for (year, month), group in df_silver.groupby(['_year', '_month']):
        partition_dir = output_dir / 'source=reddit' / f'year={year}' / f'month={month:02d}'
        partition_dir.mkdir(parents=True, exist_ok=True)

        # Drop internal columns before export
        df_export = group.drop(columns=['_ts', '_year', '_month'])

        filepath = partition_dir / 'sentiment_cleaned.parquet'
        df_export.to_parquet(filepath, index=False, engine='pyarrow')

        key = f"{year}-{month:02d}"
        exported[key] = filepath
        print(f"\u2713 Exported {key}: {len(df_export)} records \u2192 {filepath.relative_to(output_dir)}")

    # Also export a single consolidated CSV for easy inspection
    df_silver_clean = df_silver.drop(columns=['_ts', '_year', '_month'])
    csv_path = output_dir / 'reddit_sentiment_consolidated.csv'
    df_silver_clean.to_csv(csv_path, index=False)
    print(f"\n\u2713 Consolidated CSV: {csv_path.name} ({len(df_silver_clean)} records)")

    return exported


# Execute export
print("=" * 80)
print("EXPORTING TO SILVER LAYER")
print("=" * 80)

exported = export_to_silver_sentiment(df_posts, PROCESSED_DIR)

print(f"\n\u2713 All Reddit sentiment data exported to {PROCESSED_DIR}")
print(f"\u2713 {len(exported)} partition(s) written")

In [ ]:
# Verify exported files
print("\n" + "=" * 80)
print("VERIFICATION: Silver Layer Schema Compliance")
print("=" * 80)

expected_columns = [
    'timestamp_utc', 'article_id', 'pair', 'headline',
    'sentiment_score', 'sentiment_label', 'document_type',
    'speaker', 'source', 'url'
]

for key, path in exported.items():
    print(f"\nPartition {key}:")
    df_verify = pd.read_parquet(path)

    # Schema check
    actual_columns = df_verify.columns.tolist()
    if actual_columns == expected_columns:
        print("  \u2713 Schema compliant")
    else:
        missing = set(expected_columns) - set(actual_columns)
        extra = set(actual_columns) - set(expected_columns)
        if missing:
            print(f"  \u2717 Missing columns: {missing}")
        if extra:
            print(f"  \u26a0 Extra columns: {extra}")

    # Content check
    print(f"  Records: {len(df_verify)}")
    print(f"  Date range: {df_verify['timestamp_utc'].min()} to {df_verify['timestamp_utc'].max()}")
    print(f"  Sources: {df_verify['source'].unique().tolist()}")
    print(f"  Pairs: {df_verify['pair'].value_counts().head(5).to_dict()}")
    print(f"  Sentiment labels: {df_verify['sentiment_label'].value_counts().to_dict()}")

    print("\n  Sample rows:")
    print(df_verify[['timestamp_utc', 'pair', 'sentiment_score', 'sentiment_label', 'headline']].head(3).to_string(index=False))

print("\n" + "=" * 80)
print("\u2713 All files verified and ready for Sentiment Agent consumption")
print("=" * 80)

---

## Summary

This notebook answered the question: *does retail trader sentiment on Reddit's r/Forex have any predictive relationship with FX pair direction or volatility?*

The answer: **the signal exists but is weak and noisy.** Reddit sentiment should be treated as a complementary input (retail positioning proxy) rather than a standalone signal. Its value increases when combined with institutional sources — if COT data shows extreme long positioning *and* Reddit sentiment is euphoric, the convergence of institutional and retail crowding strengthens the contrarian case.

**Data Source**: Arctic Shift API (`arctic-shift.photon-reddit.com`) — a free Reddit archive providing full historical access without authentication. This replaced Reddit's public JSON endpoint which now returns 403 Blocked for unauthenticated requests.

**Sentiment Model**: FinBERT (`ProsusAI/finbert`) — the same BERT model fine-tuned on financial text used across the project. Scores are continuous in [-1.0, 1.0] with three label categories (positive, negative, neutral).

**Outputs**:
- Partitioned Parquet files in `data/processed/sentiment/source=reddit/year={YYYY}/month={MM}/`
- Consolidated CSV at `data/processed/sentiment/reddit_sentiment_consolidated.csv`
- All files follow the Silver Sentiment schema: `[timestamp_utc, article_id, pair, headline, sentiment_score, sentiment_label, document_type, speaker, source, url]`

**Limitations**:
- Arctic Shift returns max 100 posts per request; pagination is handled by sliding the `after` cursor
- Scores and comment counts may lag by ~36 hours for recent posts
- FinBERT has a 512-token input limit; very long posts are truncated. It also cannot reliably detect irony or sarcasm
- The community skews toward retail traders with limited capital and experience — this is by design (we want the retail sentiment signal) but means the signal quality is fundamentally lower than institutional data sources

---
*FX-AlphaLab · W6 Data Understanding Deliverable*

---

## Glossary of Technical Terms

### Data Collection

**Arctic Shift** — A free, open archive of Reddit data maintained for researchers and moderators. It ingests Reddit's real-time firehose and makes the complete historical record searchable via a REST API at `arctic-shift.photon-reddit.com`. No API key or authentication required.

**Pushshift** — The predecessor to Arctic Shift. An academic archive of Reddit data used extensively in NLP research. Pushshift has been largely shut down; Arctic Shift is its spiritual successor using independently collected data.

**Date-cursor Pagination** — A pagination strategy where the `after` parameter is set to the `created_utc` of the last result in the previous batch. By sorting ascending, each batch picks up exactly where the previous one left off, allowing complete traversal of large date ranges.

**Subreddit** — A community within Reddit focused on a specific topic. r/Forex (reddit.com/r/Forex) is dedicated to foreign exchange trading discussion and has 800K+ members.

**Flair** — A category tag assigned to posts by the author or moderators. Provides content classification (e.g., "Technical Analysis", "Trade Idea", "Newbie", "Meme").

---

### Sentiment Analysis

**FinBERT** — A BERT language model fine-tuned on financial text by Prosus AI (`ProsusAI/finbert`). Trained on a large corpus of financial news, SEC filings, earnings calls, and analyst reports. Returns three labels (positive, negative, neutral) with a confidence score. Used project-wide — both in the `NewsPreprocessor` for central bank communications and in this notebook for Reddit sentiment — ensuring consistent, calibrated scoring across all text sources.

**Confidence Score** — FinBERT's output probability for the predicted label, ranging from 0.0 to 1.0. We convert this to a signed score: positive → +confidence, negative → −confidence, neutral → 0.0. This produces a continuous sentiment metric in [-1.0, 1.0] comparable across sources.

**BERT (Bidirectional Encoder Representations from Transformers)** — A transformer-based language model pre-trained on large text corpora. FinBERT is BERT further fine-tuned on financial domain text to understand domain-specific terminology and sentiment patterns.

**Engagement-Weighted Sentiment** — A weighted average where each post's contribution to the weekly aggregate is proportional to its upvote score. Posts that the community agrees with (upvotes) influence the aggregate more than posts that are ignored or downvoted.

---

### Signal Analysis

**Autocorrelation** — The correlation of a time series with a lagged version of itself. Positive lag-1 autocorrelation means this week's value is predictive of next week's — indicating persistence (trending). Near-zero autocorrelation means each observation is independent (noise).

**Sentiment Dispersion** — The standard deviation of sentiment scores within a time period. High dispersion indicates community disagreement; low dispersion indicates consensus. Spikes in dispersion may precede periods of elevated market volatility.

**Retail Positioning Proxy** — Reddit sentiment serves as an indirect measurement of how retail traders are positioned. Unlike institutional positioning (reported via CFTC COT), retail positioning has no official disclosure mechanism. Social media sentiment is one of the few available proxies.

**Signal-to-Noise Ratio (SNR)** — The proportion of useful information (signal) relative to random variation (noise) in a dataset. Reddit sentiment has low SNR compared to CFTC data (regulatory-grade) or central bank publications (professional authorship). Aggregation (weekly means, engagement weighting) is used to improve SNR by averaging out noise.

---

### Reddit-Specific Terms

**Score (Upvotes)** — The net vote count on a post (upvotes minus downvotes). Reflects community agreement. Reddit fuzzes exact vote counts as an anti-spam measure, but the relative ranking is reliable.

**Bronze Layer** — In the project's Medallion architecture, the raw immutable data layer. Arctic Shift JSON responses are saved here before any transformation.

**Silver Layer** — The processed, validated, schema-compliant data layer. Reddit posts are cleaned, sentiment-scored, and mapped to the standardized sentiment schema before export here.

---
*FX-AlphaLab · W6 Glossary*